In [294]:
# Copyright 2020 by Emilio Cabrera
# All rights reserved.


The MIT License (MIT)

Copyright (c) 2020 Emilio Cabrera

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
import numpy as np
from scipy.stats import norm
from timeit import default_timer as timer
from numba import jit

In [289]:
class OptionsPricingCS:#Options pricing close solution and simulations
    
    def __init__(self, call_put, excercise_type, strike, time_to_maturity, spot, rf, rd, volatility):
        self.call_put = call_put #String call or put
        self.excercise_type = excercise_type #String American or European
        self.strike = strike #Double strike price
        self.time_to_maturity = time_to_maturity #Double years to maturity
        self.spot = spot #Double spot price
        self.rf = rf #Double foreign interest rate or dividend rate
        self.rd = rd #Double risk free rate or domestic rate
        self.volatility = volatility #Volatility
        self.Nd1 = 0
        self.Nd2 = 0
        
    def european_call_BS(self): #Black-Scholes-Merton Formula for Vanilla European Call
        self.d1 = (np.log(self.spot/ self.strike) + 
                   (self.rd - self.rf + 0.5 * self.volatility**2)*self.time_to_maturity) / (self.volatility*np.sqrt(self.time_to_maturity))
        self.d2 = self.d1 - self.volatility*np.sqrt(self.time_to_maturity)
        self.N_d1 = norm.cdf(self.d1)
        self.N_d2 = norm.cdf(self.d2)
        self.call_pv = self.spot*np.exp(-self.rf*self.time_to_maturity)*self.N_d1 - self.strike*np.exp(-self.rd*self.time_to_maturity)*self.N_d2
        self.delta = np.exp(-self.rf*self.time_to_maturity)*self.N_d1
        return self.call_pv, self.delta                     
        
    def european_put_BS(self): #Black-Scholes-Merton Formula for Vanilla European Call
        self.d1 = (np.log(self.spot/ self.strike) + 
                   (self.rd - self.rf + 0.5 * self.volatility**2)*self.time_to_maturity) / (self.volatility*np.sqrt(self.time_to_maturity))
        self.d2 = self.d1 - self.volatility*np.sqrt(self.time_to_maturity)
        self.N_d1 = norm.cdf(-self.d1)
        self.N_d2 = norm.cdf(-self.d2)
        self.put_pv = - self.spot*np.exp(-self.rf*self.time_to_maturity)*self.N_d1 + self.strike*np.exp(-self.rd*self.time_to_maturity)*self.N_d2
        self.delta = - np.exp(-self.rf*self.time_to_maturity)*self.N_d1
        return self.put_pv, self.delta
    
    def european_put_call_parity(self):
        return self.european_call_BS()[0] - self.european_put_BS()[0]
    
    @jit(nopython=True) #Make it faster
    def binomial_tree(self, steps = 100):
        self.N = steps
        self.up_factor = np.exp(self.volatility * np.sqrt(self.time_to_maturity/self.N))
        self.down_factor = 1/self.up_factor
        self.tree_matrix = np.matrix(np.zeros(shape = (self.N, self.N)))
        self.tree_matrix[0,0] = self.spot
        for i in range(self.N):
            for j in range(self.N):
                if j > 0 and i == j:
                    self.tree_matrix[i,j] = self.tree_matrix[i-1,j-1]*self.down_factor
                if j > i:
                    self.tree_matrix[i,j] = self.tree_matrix[i,j-1]*self.up_factor
        return self.tree_matrix
    
    #Note on Numba: the first invocation of the function will actually make it slower
    #subsequent invocations will be faster
    @jit(nopython=True) #Make it faster
    def european_call_binomial(self, tree ,N = 100 ):
        self.tree = tree #self.binomial_tree(steps = N)
        self.steps = self.tree.shape[1]
        self.up_factor = np.exp(self.volatility * np.sqrt(self.time_to_maturity/self.steps))
        self.down_factor = 1/self.up_factor
        self.pu = (np.exp((self.rd - self.rf)/self.steps) - self.down_factor)/(self.up_factor - self.down_factor)
        self.pd = 1 - self.pu
        self.df = np.exp(-self.rd/self.steps)
        self.tree_matrix = np.matrix(np.zeros(shape = (self.steps, self.steps)))
        self.tree_matrix[:,self.steps - 1] = np.maximum(self.tree[:,self.steps - 1] - self.strike,0)
        
        for j in range(self.steps - 2, -1, -1):
            for i in range(self.steps -  1):
                self.tree_matrix[i,j] = (self.pu*self.tree_matrix[i,j+1] + self.pd*self.tree_matrix[i+1,j+1])*self.df
        
        self.delta = (self.tree_matrix[0,1] - self.tree_matrix[1,1])/(self.tree[0,1] - self.tree[1,1])
        return self.tree_matrix[0,0], self.delta
    
    @jit(nopython=True) #Make it faster
    def european_put_binomial(self,tree, N = 100 ):
        self.tree = tree #self.binomial_tree(steps = N)
        self.steps = self.tree.shape[1]
        self.up_factor = np.exp(self.volatility * np.sqrt(self.time_to_maturity/self.steps))
        self.down_factor = 1/self.up_factor
        self.pu = (np.exp((self.rd - self.rf)/self.steps) - self.down_factor)/(self.up_factor - self.down_factor)
        self.pd = 1 - self.pu
        self.df = np.exp(-self.rd/self.steps)
        self.tree_matrix = np.matrix(np.zeros(shape = (self.steps, self.steps)))
        self.tree_matrix[:,self.steps - 1] = np.maximum(- self.tree[:,self.steps - 1] + self.strike,0)
        
        for j in range(self.steps - 2, -1, -1):
            for i in range(self.steps -  1):
                self.tree_matrix[i,j] = (self.pu*self.tree_matrix[i,j+1] + self.pd*self.tree_matrix[i+1,j+1])*self.df
        
        self.delta = (self.tree_matrix[0,1] - self.tree_matrix[1,1])/(self.tree[0,1] - self.tree[1,1])       
        return self.tree_matrix[0,0], self.delta
    
    @jit(nopython=True) #Make it faster
    def european_put_call_parity_bin(self,N = 100):
        tree = self.binomial_tree(steps = N)
        return self.european_call_binomial(tree, N=N )[0] - self.european_put_binomial( tree, N=N)[0]
    
    def compare_times(self, N=100): #Under development 28 apr 2020
        """
        print("Black and Scholes call time in seconds: ")
        %time _ = self.european_call_BS()
        print("Black and Scholes put time in seconds: ")
        %time _ = self.european_put_BS()
        print("Black and Scholes call time in seconds: ")
        %time _ = self.european_call_binomial(N)
        print("Black and Scholes put time in seconds: ")
        %time _ = self.european_put_binomial(N)
        """
        %%time 
        print("Black and Scholes call time in seconds: ")
        self.european_call_BS()
        

#Declare an OptionPricingCS object call my_option

In [258]:
my_option = OptionsPricingCS("call","European",100,1,100,0.00,0.02,0.2)

Price my_option as an European Call with Black and Scholes and Delta

In [259]:
my_option.european_call_BS()

(8.916037278572539, 0.579259709439103)

Price my_option as an European Put with Black and Scholes and Delta

In [260]:
my_option.european_put_BS()


(6.93590460924807, -0.42074029056089696)

Compute my_option Put-Call parity with Black and Scholes


In [261]:
my_option.european_put_call_parity()

1.980132669324469

Print the Binomial Tree

In [262]:
my_option.binomial_tree()

matrix([[100.        , 102.020134  , 104.08107742, ..., 695.87509706,
         709.93270652, 724.27429852],
        [  0.        ,  98.01986733, 100.        , ..., 668.58944423,
         682.09584693, 695.87509706],
        [  0.        ,   0.        ,  96.07894392, ..., 642.37367714,
         655.35048622, 668.58944423],
        ...,
        [  0.        ,   0.        ,   0.        , ...,  14.37039498,
          14.66069621,  14.95686192],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
          14.08584209,  14.37039498],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,  13.80692373]])

Fair value of the call with the binomial model and delta

In [263]:
my_option.european_call_binomial(my_option.binomial_tree())

(8.886446501926175, 0.5786703340393738)

Fair value of the put with binomial model and delta

In [264]:
my_option.european_put_binomial(my_option.binomial_tree())

(6.925919766595378, -0.4213296659606335)

Fair value of the put-call parity with the binomial tree

In [274]:
my_option.european_put_call_parity_bin(500)

1.9762117962190526

In [291]:
my_option.compare_times()

Black and Scholes call time in seconds: 
0.0003921000006812392
Black and Scholes put time in seconds: 
0.0005604999996648985
